In [ ]:
import numpy as np
import re
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot

In [ ]:
def plot_imagematch(phot, fname, fname_worse_seeing, star_coords):

    m = re.match("science_reduced_img\.(.*?)\.(\d*?)\.(.*?)\.chip1\.(\d*?)\.fits", fname_worse_seeing)
    mjd_ref = m.group(2)
    ichip_ref = m.group(4)

    orig_fname    = re.sub(".fits", ".bkg.fits", fname)
    blurred_fname = re.sub(".fits",".bkg_conv_{}_{}.fits".format(mjd_ref,ichip_ref),fname)
    target_fname  = re.sub(".fits", ".bkg.fits", fname_worse_seeing)

    original = fits.getdata("bkg/"+orig_fname)
    blurred  = fits.getdata("bkg/"+blurred_fname)
    target   = fits.getdata("bkg/"+target_fname)
    diff     = target-blurred

    #BHD
    #e image
    iy1_be = 0  + 40 #phot.ix1_z-40
    iy2_be = 70 + 40 #phot.ix2_z-40
    ix1_be = 685 + 300 #phot.iy1_z-300
    ix2_be = 755 + 300 #phot.iy2_z-300

    #Star used for seeing. 
    iy1_se = int(star_coords[1]) - 35 #- 40
    iy2_se = int(star_coords[1]) + 35 #- 40
    ix1_se = int(star_coords[0]) - 35 #- 300
    ix2_se = int(star_coords[0]) + 35 #- 300


    fig, axs = plt.subplots(4, 3, figsize=(24,35))
    for ax in axs.flatten():
        ax.set_xticks([])
        ax.set_yticks([])

    for i in range(axs.shape[0]):

        if i<2:
            iy1, iy2, ix1, ix2 = (iy1_be, iy2_be, ix1_be, ix2_be)
            obj = "BHD"
        else:
            iy1, iy2, ix1, ix2 = (iy1_se, iy2_se, ix1_se, ix2_se)
            obj = "Star"
        beam = "e-beam"
        if i==1 or i==3:
            beam = "o-beam"
            iy1 += 90 #int(phot.dy_use)
            iy2 += 90 #int(phot.dy_use)

        norm_ims = ImageNormalize(original[iy1:iy2, ix1:ix2], stretch=LinearStretch(), interval=ZScaleInterval())
        norm_diff = ImageNormalize(diff[iy1:iy2, ix1:ix2], stretch=LinearStretch(), interval=ZScaleInterval())
        axs[i,0].imshow(original[iy1:iy2, ix1:ix2], norm=norm_ims, cmap='gray')
        axs[i,1].imshow(blurred[iy1:iy2, ix1:ix2], norm=norm_ims, cmap='gray')
        axs[i,2].imshow(diff[iy1:iy2, ix1:ix2], norm=norm_ims, cmap='gray')

        axs[i,0].text(0.1, 0.9, obj , fontsize=36, weight='bold', color='white', transform=axs[i,0].transAxes)
        axs[i,0].text(0.1, 0.8, beam, fontsize=32, weight='bold', color='white', transform=axs[i,0].transAxes)
            

    fig.suptitle(fname, fontsize=36, weight='bold')

    plt.tight_layout()
    plt.show()

    return 


In [ ]:
def run_all(obj, band, star_coords, stddev_0_vals=None, bhd_coords=(1021.5, 70.), show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, special_ob_pairs=None, only_seeing_fit=False, centroid_box_size=5, pdata_filenames_for_masks=None):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #BHD coords
    bhd_ex_ref, bhd_ey_ref = bhd_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)
    #Now, go through all the OB pairs.
    ob_combs = list()
    for ob_pair in bhd.ob_pairs:
        ob_combs.append([[ob_pair[0]],[ob_pair[1]]])
    if len(bhd.ob_pairs)>1:
        ob_combs.append([None, None])

    #Add special combinations if provided. 
    if special_ob_pairs is not None:
        ob_combs.extend(special_ob_pairs)

    for k, ob_comb in enumerate(ob_combs):
        ob_ids = ob_comb[0]
        mjds = ob_comb[1]
        #print(ob_ids, mjds)
        #continue

        #Get the seeing. 
        fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
        if stddev_0_vals is not None:
            stddev_0 = stddev_0_vals[k]
        else:
            stddev_0 = np.ones(len(fnames))*1.1
        phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=ob_ids, mjds=mjds)
        #print(phot.seeing)
        if only_seeing_fit:
            continue

        #Find the image with the worst seeing. 
        iworse = np.argmax(phot.seeing)

        print("#################")
        print(ob_ids, mjds)
        print("Worst seeing image: ", fnames[iworse])
        print("Seeing: ", phot.seeing[iworse])

        for i, fname in enumerate(fnames):
            if i==iworse:
                continue
            plot_imagematch(phot, fname, fnames[iworse], star_coords)

    return
        


In [ ]:
objects = [
    "W0019-1046",
    "W0116-0505",
    "W0204-0506",
    "W0220+0137",
    "W0831+0140",
]
bands = dict()
for obj in objects:
    bands[obj] = ["R_SPECIAL"]
    if obj=="W0116-0505":
        bands[obj].extend(["I_BESS","v_HIGH"])

In [ ]:
star_coords = dict()
cat = open("reference_stars.txt")
for line in cat:
    x = line.split()
    star_coords[x[0]] = (float(x[1]), float(x[2]))
cat.close()

In [ ]:
for obj in objects:
    for band in bands[obj]:
        #run_all(obj, band, star_coords=star_coords[obj])